In [1]:
import numpy as np
from astropy.io import fits
from astropy import stats, units as u
import os 
from tqdm import tqdm
from astropy.table import QTable, hstack, Column
import matplotlib.pyplot as plt

In [2]:
# Define names and filenames...
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/'

In [3]:
def get_files(endswith, root_dir): 

    file_list = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:

            if 'mosaic' in root:
                continue
            if '550' in root:
                continue
            if '547' in root:
                continue

            if file.endswith(endswith):
                print(os.path.join(root, file))
                file_list += [os.path.join(root, file)]

    print('TOTAL NUMBER OF FILES:', len(file_list))
    return(file_list)

files_maps = get_files('_hst_ha_as2.fits', root_dir)
files_errmaps_ = get_files('_hst_ha_err_as2.fits', root_dir)
files_nebs_ = get_files('_nebmask.fits', root_dir)

/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc3627/hst_contsub/ngc3627_hst_ha_as2.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4254/hst_contsub/ngc4254_hst_ha_as2.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1512/hst_contsub/ngc1512_hst_ha_as2.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1433/hst_contsub/ngc1433_hst_ha_as2.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc5068/hst_contsub/ngc5068_hst_ha_as2.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4321/hst_contsub/ngc4321_hst_ha_as2.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1365n/hst_contsub/ngc1365n_hst_ha_as2.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4303/hst_contsub/ngc4303_hst_ha_as2.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1087/hst_contsub/ngc1087_hst_ha_as2.fits
/Users/abarnes/Dropbox/work/Smallprojects/ga

In [4]:
files_nebs = []
files_errmaps = []

for file_maps in files_maps: 

    for file_nebs in files_nebs_: 
        if file_maps.split(root_dir)[-1].split('/')[0] == file_nebs.split(root_dir)[-1].split('/')[0]:
            
            if file_maps.split('/')[-1].split('_')[0].lower() != file_nebs.split('/')[-1].split('_')[0].lower():
                print('ERROR: MISMATCHED FILE NAMES')
                print(file_maps.split('/')[-1], file_nebs.split('/')[-1])
            files_nebs += [file_nebs]
            break

    for file_errmaps in files_errmaps_: 
            if file_maps.split(root_dir)[-1].split('/')[0] == file_errmaps.split(root_dir)[-1].split('/')[0]:
                
                if file_maps.split('/')[-1].split('_')[0].lower() != file_errmaps.split('/')[-1].split('_')[0].lower():
                    print('ERROR: MISMATCHED FILE NAMES')
                    print(file_maps.split('/')[-1], file_nebs.split('/')[-1])
                files_errmaps += [file_errmaps]
                break

ERROR: MISMATCHED FILE NAMES
ngc1365n_hst_ha_as2.fits NGC1365_nebmask.fits
ERROR: MISMATCHED FILE NAMES
ngc2835s_hst_ha_as2.fits NGC2835_nebmask.fits
ERROR: MISMATCHED FILE NAMES
ngc628e_hst_ha_as2.fits NGC0628_nebmask.fits
ERROR: MISMATCHED FILE NAMES
ngc628c_hst_ha_as2.fits NGC0628_nebmask.fits


In [5]:
def get_stats(data): 

    data = data[~np.isnan(data)]
    data = data[data!=0]
    data = data[np.isfinite(data)]

    mean = np.nanmean(data)
    median = np.nanmedian(data)
    p = np.nanpercentile(data, [5, 16, 84, 95])
    std = stats.mad_std(data, ignore_nan=True) 
    return([mean, median, p[0], p[1], p[2], p[3], std])

gals  = []
output_stats = []
output_stats_dict = []

for i in tqdm(range(len(files_maps)), desc='Loading files...'): 

    file_maps, file_errmaps = files_maps[i], files_errmaps[i]

    # Loading files...
    hstha_hdu = fits.open(file_maps)[0]
    hsthaerr_hdu = fits.open(file_errmaps)[0]

    factor = 1e3
    data_map = hstha_hdu.data / factor
    data_err = hsthaerr_hdu.data / factor

    # Get RMS for whole map... 
    stats_map = get_stats(data_map)
    stats_err = get_stats(data_err)

    gal = file_maps.split('/')[-1].split('_hst_ha_as2.fits')[0].upper()
    gals += [gal]

    output_stats += [[stats_map+stats_err]]
    output_stats_dict += [{'galaxy': gal, 'stats_map': stats_map, 'stats_err': stats_err}]

gals = np.array(gals, dtype='<U12')
output_stats = np.array(output_stats)

gals

Loading files...: 100%|██████████| 20/20 [00:56<00:00,  2.84s/it]


array(['NGC3627', 'NGC4254', 'NGC1512', 'NGC1433', 'NGC5068', 'NGC4321',
       'NGC1365N', 'NGC4303', 'NGC1087', 'NGC7496', 'NGC1385', 'NGC2835S',
       'IC5332', 'NGC628E', 'NGC3351', 'NGC628C', 'NGC1672', 'NGC4535',
       'NGC1300', 'NGC1566'], dtype='<U12')

In [6]:
# Get into a astropy table the output of the stats of the maps to .tex

gal_names = gals.copy()
for i, gal in enumerate(gal_names):
    gal = gal.upper()
    if 'NGC' in gal:
        gal_split = gal.split('NGC')
        gal = 'NGC~' + gal_split[1]
    if 'IC' in gal:
        gal_split = gal.split('IC')
        gal = 'IC~' + gal_split[1]
        
    if '628C' in gal:
        gal_split = gal.split('628C')
        gal = gal_split[0] + '0628-C'
    if '628E' in gal:
        gal_split = gal.split('628E')
        gal = gal_split[0] + '0628-E'

    gal_names[i] = gal

names=('mean_map', 'median_map', 'p5_map', 'p16_map', 'p84_map', 'p95_map', 'std_map', 
        'mean_noise', 'median_noise', 'p5_noise', 'p16_noise', 'p84_noise', 'p95_noise', 'std_noise')

table1 = Column(gal_names, name='gal_name')
table2 = QTable(np.array(output_stats)[:,0,:], names=names)

table = hstack([table1, table2])
table.sort('gal_name')
table

gal_name,mean_map,median_map,p5_map,p16_map,p84_map,p95_map,std_map,mean_noise,median_noise,p5_noise,p16_noise,p84_noise,p95_noise,std_noise
str12,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
IC~5332,6.470802307128906,3.99265193939209,-92.39365615844727,-54.109260406494144,64.58054321289062,107.98428573608398,59.323578208779416,77.87606811523438,73.41838073730469,66.49895477294922,69.1404800415039,80.44386291503906,93.2486156463623,4.944199913100493
NGC~0628-C,15.354203224182129,8.892777442932129,-31.417768001556396,-15.403205871582031,36.801689300537106,64.14998741149901,25.5759176027321,43.29826354980469,41.386444091796875,36.739149284362796,38.48522186279297,46.19519805908203,55.09638404846186,3.398016555448316
NGC~0628-E,5.8939924240112305,3.2863171100616455,-99.31922302246093,-58.28450393676758,67.63097167968749,114.74555053710935,62.822316744930575,82.08067321777344,77.16777801513672,69.85974960327148,72.69075775146484,85.00141113281249,101.41996002197266,5.152153560608094
NGC~1087,18.38296127319336,9.88846206665039,-92.01887588500976,-51.77456222534179,77.76969757080077,136.09093170165997,64.1406837117303,82.11289978027344,75.35559844970703,66.99991760253906,69.94557189941406,85.50409698486328,99.4819625854492,6.747111490756276
NGC~1300,4.006114482879639,2.4126625061035156,-22.462682723999023,-12.464393196105958,18.184948501586916,30.68775177001953,15.219769117283366,27.634923934936523,26.426128387451172,23.591593265533447,24.65253257751465,29.006336212158203,33.95276737213135,2.0448190918122413
NGC~1365N,29.851825714111328,13.878876686096191,-79.59961700439453,-42.39872283935547,76.42736663818354,135.0986785888672,58.5954435193748,74.98429107666016,68.8123779296875,60.925985717773436,63.62876510620117,79.58569091796875,95.6545501708984,6.521302865817102
NGC~1385,18.634401321411133,7.965470790863037,-93.7087173461914,-53.40602493286133,76.01522979736329,138.41110229492188,63.881551828309405,80.2713394165039,74.06060791015625,65.8154067993164,68.80245208740234,84.77040100097656,100.7823669433593,6.478840030670999
NGC~1433,13.972528457641602,3.931461811065674,-92.44430351257324,-53.803930053710936,64.706787109375,110.5165786743164,59.08528184542754,82.1244125366211,75.70110321044922,66.61870574951172,69.96965789794922,86.33190155029297,102.84809875488281,7.030370499208251
NGC~1512,5.26482629776001,2.272935390472412,-102.50092430114746,-60.27679214477539,67.26407745361328,114.51224098205563,63.655941152122246,84.80207824707031,77.14189147949219,69.07229461669922,72.12133026123047,85.87727355957031,102.13651657104491,5.845449269658428


In [7]:
formats={
        'gal_name': '%s',
        'mean_map': '%.1f',
        'median_map': '%.1f',
        'p5_map': '%.1f',
        'p16_map': '%.1f',
        'p84_map': '%.1f',
        'p95_map': '%.1f',
        'std_map': '%.1f',
        'mean_noise': '%.1f',
        'median_noise': '%.1f',
        'p5_noise': '%.1f',
        'p16_noise': '%.1f',
        'p84_noise': '%.1f',
        'p95_noise': '%.1f',
        'std_noise': '%.1f'
        }

table.write('./tabs/galmap_props.tex', formats=formats, overwrite=True)

In [8]:
# Reorganise to match names in .fits files
gals = table['gal_name']

for i, gal in enumerate(gals):
    if '~' in gal:
        gal_split = gal.split('~')
        gal = gal_split[0] + gal_split[1]
    table['gal_name'][i] = gal

table.write('./tabs/galmap_props.fits', overwrite=True)
table

gal_name,mean_map,median_map,p5_map,p16_map,p84_map,p95_map,std_map,mean_noise,median_noise,p5_noise,p16_noise,p84_noise,p95_noise,std_noise
str12,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
IC5332,6.470802307128906,3.99265193939209,-92.39365615844727,-54.109260406494144,64.58054321289062,107.98428573608398,59.323578208779416,77.87606811523438,73.41838073730469,66.49895477294922,69.1404800415039,80.44386291503906,93.2486156463623,4.944199913100493
NGC0628-C,15.354203224182129,8.892777442932129,-31.417768001556396,-15.403205871582031,36.801689300537106,64.14998741149901,25.5759176027321,43.29826354980469,41.386444091796875,36.739149284362796,38.48522186279297,46.19519805908203,55.09638404846186,3.398016555448316
NGC0628-E,5.8939924240112305,3.2863171100616455,-99.31922302246093,-58.28450393676758,67.63097167968749,114.74555053710935,62.822316744930575,82.08067321777344,77.16777801513672,69.85974960327148,72.69075775146484,85.00141113281249,101.41996002197266,5.152153560608094
NGC1087,18.38296127319336,9.88846206665039,-92.01887588500976,-51.77456222534179,77.76969757080077,136.09093170165997,64.1406837117303,82.11289978027344,75.35559844970703,66.99991760253906,69.94557189941406,85.50409698486328,99.4819625854492,6.747111490756276
NGC1300,4.006114482879639,2.4126625061035156,-22.462682723999023,-12.464393196105958,18.184948501586916,30.68775177001953,15.219769117283366,27.634923934936523,26.426128387451172,23.591593265533447,24.65253257751465,29.006336212158203,33.95276737213135,2.0448190918122413
NGC1365N,29.851825714111328,13.878876686096191,-79.59961700439453,-42.39872283935547,76.42736663818354,135.0986785888672,58.5954435193748,74.98429107666016,68.8123779296875,60.925985717773436,63.62876510620117,79.58569091796875,95.6545501708984,6.521302865817102
NGC1385,18.634401321411133,7.965470790863037,-93.7087173461914,-53.40602493286133,76.01522979736329,138.41110229492188,63.881551828309405,80.2713394165039,74.06060791015625,65.8154067993164,68.80245208740234,84.77040100097656,100.7823669433593,6.478840030670999
NGC1433,13.972528457641602,3.931461811065674,-92.44430351257324,-53.803930053710936,64.706787109375,110.5165786743164,59.08528184542754,82.1244125366211,75.70110321044922,66.61870574951172,69.96965789794922,86.33190155029297,102.84809875488281,7.030370499208251
NGC1512,5.26482629776001,2.272935390472412,-102.50092430114746,-60.27679214477539,67.26407745361328,114.51224098205563,63.655941152122246,84.80207824707031,77.14189147949219,69.07229461669922,72.12133026123047,85.87727355957031,102.13651657104491,5.845449269658428


In [9]:
# def get_hist(data, bins='', nbins=50, logbins=False, norm=True, cum=False):

#     """Get histogram

#     Parameters
#     ----------
#     data : np.array
#         input data
#     bins : list
#         input bin edges for histogram calculaiton; default=''
#     nbins : int
#         number of bins to determine if bins is not given; defult=50
#     logbins : bool
#         logarithmically spaced bins if bins is not given
#     norm : bool
#         normalise such that max is equal to unity; default=True
#     cum : bool
#         cumulative distorbution; otherwise probability distorbution
#     Returns
#     -------
#     bins : list
#         bin edges for histogram calculaiton
#     bin_cent : np.array
#         bin centres, for easy plotting in matplotlib
#     hist : np.array
#         histogram data for each bin centre
#     """
    
#     data = data.flatten()

#     if bins.all() is None:
#         vmin=np.nanmin(data)
#         vmax=np.nanmax(data)

#         bmin = vmin - (np.absolute(vmin)*1)
#         bmax = vmax + (np.absolute(vmax)*0.3)

#         if logbins:
#             min = np.nanmin(data[data>0])
#             bins = np.logspace(np.log10(bmin), np.log10(bmax), nbins+1)
#         else:
#             bins = np.linspace(bmin, bmax, nbins+1)
#     else:
#         nbins = len(bins)-1

#     bins_cent = np.empty([nbins])

#     for i in range(nbins):
#         bins_cent[i] = np.nanmean([bins[i], bins[i+1]])

#     hist = np.histogram(data.flatten(), bins)[0]

#     if cum:
#         hist = np.cumsum(hist)
#     if norm:
#         hist = hist/np.nanmax(hist)

#     return(bins, bins_cent, hist)

In [10]:
# # for i in tqdm(range(len(files_maps)), desc='Loading files...'): 
# for i in [0]: 

#     file_maps, file_errmaps = files_maps[i], files_errmaps[i]

#     # Loading files...
#     hstha_hdu = fits.open(file_maps)[0]
#     hsthaerr_hdu = fits.open(file_errmaps)[0]

#     factor = 1e3
#     pix_size_hst = (hstha_hdu.header['CD1_1']*u.deg).to('arcsec')
#     pix_area_hst = pix_size_hst**2

#     data_map = hstha_hdu.data / pix_area_hst.value / factor
#     data_err = hsthaerr_hdu.data / pix_area_hst.value / factor

#     # # Get RMS for whole map... 
#     stats_map = get_stats(data_map)
#     stats_err = get_stats(data_err)

#     bins = np.linspace(-300, 300, 100)
#     bins, bins_cent, hist = get_hist(data_map, bins)

In [11]:
# err_mean = stats_err[0]
# map_std = stats_map[-1]
# map_mean = stats_map[1]

In [12]:
# color = f'C{i}'
# plt.plot(bins_cent, hist, ds='steps-mid', c=color)
# plt.fill_between(bins_cent, hist, y2=0, step="mid", alpha=0.2, zorder=0, fc=color, ec='none')

# # plt.plot(stats_map[0], 0, 'ro')
# plt.plot([map_mean-err_mean, map_mean+err_mean], [0.1, 0.1], c=color, ls='--')
# plt.plot([map_mean-map_std, map_mean+map_std], [0.05, 0.05], c=color, ls=':')